# Environment setup

In [46]:
# ! python -m pip install detectron2 -f \
#   https://dl.fbaipublicfiles.com/detectron2/wheels/cu113/torch1.10/index.html

In [1]:
%cd /workspace/GitHub/AVSL/models

/workspace/GitHub/AVSL/models


In [2]:
import detectron2.utils.comm as comm
from detectron2.config import get_cfg
from detectron2.projects.deeplab import add_deeplab_config
from detectron2.utils.logger import setup_logger

# from mask2former.config import add_maskformer2_config
from Mask2former.mask2former import add_maskformer2_config

- init.py: from . import modeling
- modeling/init.py: from .pixel_decoder.msdeformattn import MSDeformAttnPixelDecoder
- modeling/pixel_decoder/msdeformattn.py: from .ops.modules import MSDeformAttn
- modeling/pixel_decoder/ops/modules/ms_deform_attn.py: from ..functions import MSDeformAttnFunction
- modeling/pixel_decoder/ops/functions/ms_deform_attn_func.py: import MultiScaleDeformableAttention as MSDA 주석 처리

In [3]:
# Create configs and perform basic setups

cfg = get_cfg()
# for poly lr schedule
add_deeplab_config(cfg)
add_maskformer2_config(cfg)
cfg.merge_from_file("Mask2former/configs/custom/MaskAVSL_swin_large.yaml")
# cfg.merge_from_list(args.opts)
cfg.MODEL.DEVICE = "cuda:1"
cfg.eval_only = True
cfg.freeze()
# default_setup(cfg, args)
# Setup logger for "mask_former" module
setup_logger(output=cfg.OUTPUT_DIR, distributed_rank=comm.get_rank(), name="mask2former")

<Logger mask2former (DEBUG)>

# Custom Dataset

In [4]:
import sys, os
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from moviepy.editor import VideoFileClip

from detectron2.config import configurable
from detectron2.data import transforms as T
from detectron2.projects.point_rend import ColorAugSSDTransform
from detectron2.data import detection_utils as utils

In [5]:
class VideoDataset(Dataset):
    @configurable
    def __init__(self, is_train=True, *, augmentations, image_format, ignore_label, size_divisibility, folder_path, ):
        self.folder_path = folder_path
        self.video_list = [f for f in os.listdir(folder_path) if f.endswith('.mp4')]
        self.tfm_gens = augmentations

        self.is_train = is_train
        self.img_format = image_format
        self.ignore_label = ignore_label
        self.size_divisibility = size_divisibility

    def __len__(self):
        return len(self.video_list)

    def __getitem__(self, idx):
        video_path = os.path.join(self.folder_path, self.video_list[idx])

        # Load video and extract central frame
        video_clip = VideoFileClip(video_path)
        central_frame = video_clip.get_frame(video_clip.duration / 2)
        central_frame = np.array(central_frame)
        # Convert image frame to numpy array and normalize
        #central_frame = central_frame / 255.0

        # Additional augmentation
        aug_input = T.AugInput(central_frame)
        aug_input, transforms = T.apply_transform_gens(self.tfm_gens, aug_input)
        central_frame = aug_input.image
        #sem_seg_gt = aug_input.sem_seg

        central_frame = torch.as_tensor(np.ascontiguousarray(central_frame.transpose(2, 0, 1)))
        #central_frame = np.transpose(central_frame, (2, 0, 1))  # Change HWC to CHW

        if self.size_divisibility > 0:
            central_frame_size = (central_frame.shape[-2], central_frame.shape[-1])
            padding_size = [
                0,
                self.size_divisibility - central_frame_size[1],
                0,
                self.size_divisibility - central_frame_size[0],
            ]
            central_frame = F.pad(central_frame, padding_size, value=128).contiguous()

        sample = {}
        # sample['image'] = central_frame

        # Save audio as WAV file
        audio_path = f"{video_path[:-4]}.wav"
        self.blockPrint()
        # audio = 
        video_clip.audio.write_audiofile(audio_path, codec='pcm_s16le', fps=44100)
        self.enablePrint()

        # Return data as dictionary
        sample = {'image': central_frame, 'audio_path': audio_path}
        # sample = {'image': central_frame, 'audio': audio}

        return sample

    @classmethod
    def from_config(cls, cfg, is_train=True):
        # Build augmentation
        augs = [
            T.ResizeShortestEdge(
                cfg.INPUT.MIN_SIZE_TRAIN,
                cfg.INPUT.MAX_SIZE_TRAIN,
                cfg.INPUT.MIN_SIZE_TRAIN_SAMPLING,
            )
        ]
        if cfg.INPUT.CROP.ENABLED:
            augs.append(
                T.RandomCrop_CategoryAreaConstraint(
                    cfg.INPUT.CROP.TYPE,
                    cfg.INPUT.CROP.SIZE,
                    cfg.INPUT.CROP.SINGLE_CATEGORY_MAX_AREA,
                    cfg.MODEL.SEM_SEG_HEAD.IGNORE_VALUE,
                )
            )
        if cfg.INPUT.COLOR_AUG_SSD:
            augs.append(ColorAugSSDTransform(img_format=cfg.INPUT.FORMAT))
        augs.append(T.RandomFlip())
        augs.extend([
        T.ResizeScale(
            min_scale=cfg.INPUT.MIN_SCALE, max_scale=cfg.INPUT.MAX_SCALE,
            target_height=cfg.INPUT.IMAGE_SIZE, target_width=cfg.INPUT.IMAGE_SIZE
        ),
        T.FixedSizeCrop(crop_size=(cfg.INPUT.IMAGE_SIZE, cfg.INPUT.IMAGE_SIZE)),
        ])

        ignore_label = False

        ret = {
            "is_train": is_train,
            "augmentations": augs,
            "image_format": cfg.INPUT.FORMAT,
            "ignore_label": ignore_label,
            "size_divisibility": cfg.INPUT.SIZE_DIVISIBILITY,
        }
        return ret
    
    def blockPrint(self):
        global backupstdout
        backupstdout=sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def enablePrint(self):
        global backupstdout
        sys.stdout = backupstdout

In [6]:
folder_path = '../DATA/videos'
video_dataset = VideoDataset(cfg, folder_path=folder_path)
dataloader = DataLoader(video_dataset, batch_size=cfg.SOLVER.IMS_PER_BATCH, shuffle=True)

In [7]:
print(len(video_dataset))
print(len(dataloader))

23442
1466


In [8]:
# detectron2/engine/train_loop.py

data = next(iter(dataloader))

In [9]:
# maskformer_model.py
from detectron2.structures import ImageList
from tqdm import tqdm

pixel_mean = cfg.MODEL.PIXEL_MEAN
pixel_std = cfg.MODEL.PIXEL_STD
size_divisibility = cfg.MODEL.MASK_FORMER.SIZE_DIVISIBILITY
device = cfg.MODEL.DEVICE

pixel_mean = torch.Tensor(pixel_mean).view(-1, 1, 1)
pixel_std = torch.Tensor(pixel_std).view(-1, 1, 1)

# images = [x["image"].to(device) for x in tqdm(data)]
images = [x for x in data['image']]
images = [(x - pixel_mean) / pixel_std for x in tqdm(images)]
images = ImageList.from_tensors(images, size_divisibility)

100%|██████████| 16/16 [00:00<00:00, 107.33it/s]


### show

In [56]:
# images = []
# for i, x in tqdm(enumerate(dataloader)): 
#     img = x["image"]
#     img = (img - pixel_mean) / pixel_std
#     images.append(img)
#     if i == 10:
#         break # 11개에 2분/이었는데 6분으로 늘어남 
# images = ImageList.from_tensors(images, size_divisibility)
# images.to(device)

In [57]:
# detectron2/modeling/meta_arch/build.py
# def build_model(cfg):
#     """
#     Build the whole model architecture, defined by ``cfg.MODEL.META_ARCHITECTURE``.
#     Note that it does not load any weights from ``cfg``.
#     """
#     meta_arch = cfg.MODEL.META_ARCHITECTURE
#     model = META_ARCH_REGISTRY.get(meta_arch)(cfg) # 'MaskFormer'
#     model.to(torch.device(cfg.MODEL.DEVICE))
#     _log_api_usage("modeling.meta_arch." + meta_arch)
#     return model
# -> model = maskformer_model.py

# detectron2/engine/defaults.py
# data_loader = self.build_train_loader(cfg) -> torchdata.DataLoader(dataset, batch_size=batch_size,)
# _trainer = (AMPTrainer if cfg.SOLVER.AMP.ENABLED else SimpleTrainer)(model, data_loader, optimizer)

# detectron2/engine/train_loop.py
# def _data_loader_iter(self):
#     # only create the data loader iterator when it is used
#     if self._data_loader_iter_obj is None:
#         self._data_loader_iter_obj = iter(self.data_loader)
#     return self._data_loader_iter_obj
# data = next(self._data_loader_iter) -> next(iter(self.data_loader)) 
# model(data)

In [58]:
# example = video_dataset.__getitem__(1)
# print(example.keys())

In [59]:
# example['image'].shape

In [60]:
# example['audio_path']

In [61]:
# from matplotlib import pyplot as plt
# plt.imshow(example['image'].permute(1, 2, 0), interpolation='nearest')
# plt.show()

In [62]:
# from scipy.io import wavfile

# fs, audio_data = wavfile.read(example['audio_path'])
# print(fs, audio_data.shape)

# # plt.figure(figsize = (12, 3))
# plt.plot(audio_data, lw = 1)
# plt.xlim(0, len(audio_data))

# Model Architecture

## backbone

In [63]:
%cd /workspace/GitHub/AVSL/models

/workspace/GitHub/AVSL/models


In [64]:
# from detectron2.modeling import build_backbone

# backbone = build_backbone(cfg)
# backbone = BACKBONE_REGISTRY.get(backbone_name)(cfg, input_shape)

In [65]:
# from detectron2.layers import ShapeSpec
# from Mask2former.mask2former.modeling.backbone.swin import D2SwinTransformer

# print(cfg.MODEL.WEIGHTS)
# input_shape = ShapeSpec(channels=len(cfg.MODEL.PIXEL_MEAN))
# backbone = D2SwinTransformer(cfg, input_shape)
# backbone.init_weights(cfg.MODEL.WEIGHTS)
# # backbone.to(device)

# features = backbone(images.tensor)

In [66]:
# model.py
from detectron2.layers import ShapeSpec
from Mask2former.mask2former.modeling.backbone.swin import D2SwinTransformer

def build_swin_backbone(cfg):
    input_shape = ShapeSpec(channels=len(cfg.MODEL.PIXEL_MEAN))
    model = D2SwinTransformer(cfg, input_shape)
    model.init_weights(cfg.MODEL.WEIGHTS)
    return model
backbone = build_swin_backbone(cfg)
image_feature = backbone(images.tensor)

In [67]:
image_feature.keys()

dict_keys(['res2', 'res3', 'res4', 'res5'])

In [68]:
print(image_feature['res2'].shape)
print(image_feature['res3'].shape)
print(image_feature['res4'].shape)
print(image_feature['res5'].shape)

torch.Size([16, 192, 256, 256])
torch.Size([16, 384, 128, 128])
torch.Size([16, 768, 64, 64])
torch.Size([16, 1536, 32, 32])


## pixel_decoder

In [69]:
# from detectron2.modeling import build_sem_seg_head

# sem_seg_head = build_sem_seg_head(cfg, backbone.output_shape())
# -> SEM_SEG_HEADS_REGISTRY.get(name)(cfg, input_shape)

In [70]:
# from mask2former.maskformer_model.modeling.pixel_decoder.fpn import build_pixel_decoder

# build_pixel_decoder(cfg, input_shape)
# -> model = SEM_SEG_HEADS_REGISTRY.get(name)(cfg, input_shape)

In [71]:
backbone.output_shape()

{'res2': ShapeSpec(channels=192, height=None, width=None, stride=4),
 'res3': ShapeSpec(channels=384, height=None, width=None, stride=8),
 'res4': ShapeSpec(channels=768, height=None, width=None, stride=16),
 'res5': ShapeSpec(channels=1536, height=None, width=None, stride=32)}

In [72]:
# from mask2former.modeling.pixel_decoder.fpn import TransformerEncoderPixelDecoder

# pixel_decoder = TransformerEncoderPixelDecoder(cfg, input_shape=backbone.output_shape()) # maskformer_model.py

# # mask_foremer_head.py
# pp_embeds, transformer_encoder_features, _ = pixel_decoder.forward_features(features)

In [73]:
# model.py
from Mask2former.mask2former.modeling.pixel_decoder.fpn import TransformerEncoderPixelDecoder

def build_pixel_decoder(cfg, input_shape):
    model = TransformerEncoderPixelDecoder(cfg, input_shape)
    return model

pixel_decoder = build_pixel_decoder(cfg, backbone.output_shape())
pp_embeds, image_features, _ = pixel_decoder(image_feature)

Calling forward() may cause unpredicted behavior of PixelDecoder module.


In [74]:
image_features.shape

torch.Size([16, 256, 32, 32])

## transformer decoder

In [75]:
# # mask_former_head.py
# if cfg.MODEL.MASK_FORMER.TRANSFORMER_IN_FEATURE == "transformer_encoder":
#     transformer_predictor_in_channels = cfg.MODEL.SEM_SEG_HEAD.CONVS_DIM
# elif cfg.MODEL.MASK_FORMER.TRANSFORMER_IN_FEATURE == "pixel_embedding":
#     transformer_predictor_in_channels = cfg.MODEL.SEM_SEG_HEAD.MASK_DIM
# elif cfg.MODEL.MASK_FORMER.TRANSFORMER_IN_FEATURE == "multi_scale_pixel_decoder":  # for maskformer2
#     transformer_predictor_in_channels = cfg.MODEL.SEM_SEG_HEAD.CONVS_DIM
# else:
#     transformer_predictor_in_channels = input_shape[cfg.MODEL.MASK_FORMER.TRANSFORMER_IN_FEATURE].channels

# # "transformer_predictor": build_transformer_decoder(cfg, transformer_predictor_in_channels, mask_classification=True,)
# # -> TRANSFORMER_DECODER_REGISTRY.get(name)(cfg, in_channels, mask_classification)

In [76]:
# from mask2former.modeling.transformer_decoder.maskformer_transformer_decoder import StandardTransformerDecoder

# transformer_decoder = StandardTransformerDecoder(cfg, in_channels=transformer_predictor_in_channels, mask_classification=False)

In [77]:
# mask_former_head.py
# predictions = transformer_decoder(transformer_encoder_features, mask_features, mask=None) # mask에 뭐가 들어가야 하는지 모르겠다

In [78]:
# predictions.keys()

In [79]:
# predictions['pred_masks'].shape # scoremap
# # predictions['pred_masks']

In [80]:
# print(predictions['pred_masks'].min())
# print(predictions['pred_masks'].max())
# # 음수 값이 나오는 것을 방지하기 위해 sigmoid 함수 고려

## transformer decoder 수정

In [81]:
# maskformer_transformer_decoder_custom.py
from Mask2former.mask2former.modeling.transformer_decoder.transformer import Transformer
from Mask2former.mask2former.modeling.transformer_decoder.position_encoding import PositionEmbeddingSine
from torch import nn

N_steps = cfg.MODEL.MASK_FORMER.HIDDEN_DIM // 2
pe_layer = PositionEmbeddingSine(N_steps, normalize=True)

batch_size=cfg.SOLVER.IMS_PER_BATCH

num_queries = cfg.MODEL.MASK_FORMER.NUM_OBJECT_QUERIES
transformer = Transformer(
            d_model=cfg.MODEL.MASK_FORMER.HIDDEN_DIM,
            dropout=cfg.MODEL.MASK_FORMER.DROPOUT,
            nhead=cfg.MODEL.MASK_FORMER.NHEADS,
            dim_feedforward=cfg.MODEL.MASK_FORMER.DIM_FEEDFORWARD,
            num_encoder_layers=cfg.MODEL.MASK_FORMER.ENC_LAYERS,
            num_decoder_layers=cfg.MODEL.MASK_FORMER.DEC_LAYERS,
            normalize_before=cfg.MODEL.MASK_FORMER.PRE_NORM,
            return_intermediate_dec=cfg.MODEL.MASK_FORMER.DEEP_SUPERVISION,
        )
hidden_dim = transformer.d_model


if cfg.MODEL.MASK_FORMER.TRANSFORMER_IN_FEATURE == "transformer_encoder":
    transformer_predictor_in_channels = cfg.MODEL.SEM_SEG_HEAD.CONVS_DIM
elif cfg.MODEL.MASK_FORMER.TRANSFORMER_IN_FEATURE == "pixel_embedding":
    transformer_predictor_in_channels = cfg.MODEL.SEM_SEG_HEAD.MASK_DIM

In [82]:
import fvcore.nn.weight_init as weight_init
from detectron2.layers import Conv2d
from Mask2former.mask2former.modeling.transformer_decoder.maskformer_transformer_decoder_custom import MLP

learnable_input_queries = nn.Embedding(num_queries, hidden_dim)

in_channels = transformer_predictor_in_channels # mask_former_head.py
enforce_input_project = cfg.MODEL.MASK_FORMER.ENFORCE_INPUT_PROJ
deep_supervision = cfg.MODEL.MASK_FORMER.DEEP_SUPERVISION
mask_dim = cfg.MODEL.SEM_SEG_HEAD.MASK_DIM

if in_channels != hidden_dim or enforce_input_project:
    input_proj = Conv2d(in_channels, hidden_dim, kernel_size=1)
    weight_init.c2_xavier_fill(input_proj)
else:
    input_proj = nn.Sequential()
aux_loss = deep_supervision

mask_embed = MLP(hidden_dim, hidden_dim, mask_dim, 3)

In [83]:
x = image_features

In [84]:
mask = None

pos = pe_layer(x, mask)

src = x
hs, memory = transformer(input_proj(src), mask, learnable_input_queries.weight, pos)

In [85]:
# output_size = hs.shape

audio_separation_tokens = hs[-1]
token_size = hs[-1].shape
print(token_size)

torch.Size([16, 100, 128])


### after audio

In [86]:
# def audio_separation_model(mixed_audio, audio_separation_token, noise_token): # mixed_audio: [batch_size, _, _]
#     query_input = torch.cat([audio_separation_token, noise_token], dim=1) # [batch_size, num_queries+1, 128]
#     separated_audio_wav_list = []
#     # 이 아래는 바꿔야 함
#     for i in range(num_queries):
#         separated_audio_wav_list.append(torch.rand(batch_size, 1, 903, hidden_dim))
#     return mixed_audio, separated_audio_wav_list

# mixed_audio, separated_audio_wav_list = audio_separation_model(None, audio_separation_tokens, noise_token)

In [87]:
# def audio_encoder(separated_audio_wav_list):
#     separated_audio_wav_features_list = []
#     for wav in separated_audio_wav_list:
#             out = torch.rand(batch_size, 903, hidden_dim) # num_queries = len(separated_audio_wav_features_list)
#             separated_audio_wav_features_list.append(out)
#     return separated_audio_wav_features_list

# separated_audio_wav_features_list = audio_encoder(separated_audio_wav_list)
# len(separated_audio_wav_features_list)

In [88]:
# def av_feature_embed(separated_audio_wav_features_list, pooling_mode='sum'):
#     av_feature_list = ImageList.from_tensors(separated_audio_wav_features_list).tensor
#     av_feature_list = av_feature_list.permute(1, 0, 2, 3)
#     if pooling_mode == 'sum':
#         av_feature_list = av_feature_list.sum(dim=2)
#     elif pooling_mode == 'mean':
#         av_feature_list = av_feature_list.mean(dim=2)
#     elif pooling_mode == 'max':
#         av_feature_list = av_feature_list.max(dim=2).values
#     return av_feature_list

# av_feature_list = av_feature_embed(separated_audio_wav_features_list, pooling_mode='sum')
# av_feature_list.shape

In [89]:
# def audio_module(mixed_audio, query_output, noise_query, batch_size):
#     query_input = torch.cat([query_output, noise_query], dim=1)
#     out = torch.rand(batch_size, num_queries, 903, hidden_dim)
#     return out
# av_feature = audio_module(None, hs[-1], noise_query, batch_size) # torch.Size([16, 100, 128])

In [90]:
# av_feature_embed = mask_embed(av_feature_list)

NameError: name 'av_feature_list' is not defined

: 

In [ ]:
# av_feature_embed.shape

torch.Size([16, 100, 256])

In [ ]:
# scoremap = torch.einsum("bqc,bchw->bqhw", av_feature_embed, mask_features)
# scoremap.shape
# scoremap = torch.einsum("lbqc,bchw->lbqhw", av_feature_embed, mask_features)

torch.Size([16, 100, 256, 256])

In [ ]:
# from mask2former.modeling.transformer_decoder.maskformer_transformer_decoder_custom import StandardTransformerDecoder

# def build_transformer_decoder(cfg):
#     if cfg.MODEL.MASK_FORMER.TRANSFORMER_IN_FEATURE == "transformer_encoder":
#         transformer_predictor_in_channels = cfg.MODEL.SEM_SEG_HEAD.CONVS_DIM
#     elif cfg.MODEL.MASK_FORMER.TRANSFORMER_IN_FEATURE == "pixel_embedding":
#         transformer_predictor_in_channels = cfg.MODEL.SEM_SEG_HEAD.MASK_DIM
#     elif cfg.MODEL.MASK_FORMER.TRANSFORMER_IN_FEATURE == "multi_scale_pixel_decoder":  # for maskformer2
#         transformer_predictor_in_channels = cfg.MODEL.SEM_SEG_HEAD.CONVS_DIM
#     model = StandardTransformerDecoder(cfg, in_channels=transformer_predictor_in_channels, mask_classification=False)
#     return model

# transformer_decoder = build_transformer_decoder(cfg)
# outputs = transformer_decoder(transformer_encoder_features, mask_features, mixed_audio=None, mask=None)

In [ ]:
# outputs.keys()

dict_keys(['pred_masks'])

# LAAS

In [ ]:
%cd /workspace/GitHub/AVSL/models

/workspace/GitHub/AVSL/models


In [ ]:
import librosa

audio_list = []
for path in data['audio_path']:
    y, sr = librosa.load(path, mono=True, sr=16000)
    audio_list.append(torch.from_numpy(y).unsqueeze(1))

y = ImageList.from_tensors(audio_list).tensor
y.shape

torch.Size([16, 160640, 1])

In [ ]:
audio_separation_tokens.shape

torch.Size([16, 100, 128])

In [ ]:
noise_token = torch.rand(token_size[0], 1, token_size[2])
input_tokens = torch.cat([audio_separation_tokens, noise_token], dim=1)
input_tokens.shape

torch.Size([16, 101, 128])

In [ ]:
# x = {'audio':torch.from_numpy(y.reshape((2,y.shape[1],1))),
#      'conditions':torch.from_numpy(np.zeros((2,9,128),dtype=np.float32))} # (batch_size, 오디오길이, 1), mono로 바꾸어야 함

input = {"audio": y, "conditions": input_tokens}

In [ ]:
from AudioModule.LAAS import LAAS
from AudioModule import config

laas = LAAS(config=config)
# laas(input) # kernel crashed

: 

In [ ]:
input["conditions"].shape[0]

16

In [ ]:
# pytorch-lightning: pip install pytorch-lightning==1.4.4, pip install torchmetrics==0.6.0
# from AudioModule.LAAS import ZeroShotASP
# from AudioModule import config

# ASP = ZeroShotASP(config=config, channels=input["conditions"].shape[1])

# x = ASP(input=input["audio"], condition=input["conditions"])

: 

In [ ]:
mask_pred_results = F.interpolate(
                mask_pred_results,
                size=(images.tensor.shape[-2], images.tensor.shape[-1]),
                mode="bilinear",
                align_corners=False,
            )

In [ ]:
image_size = images.image_sizes

In [ ]:
def semantic_inference(mask_cls, mask_pred):
    mask_cls = F.softmax(mask_cls, dim=-1)[..., :-1]
    mask_pred = mask_pred.sigmoid()
    semseg = torch.einsum("qc,qhw->chw", mask_cls, mask_pred)
    return semseg

def panoptic_inference(mask_cls, mask_pred):
    scores, labels = F.softmax(mask_cls, dim=-1).max(-1)
    mask_pred = mask_pred.sigmoid()

    keep = labels.ne(sem_seg_head.num_classes) & (scores > object_mask_threshold)
    cur_scores = scores[keep]
    cur_classes = labels[keep]
    cur_masks = mask_pred[keep]
    cur_mask_cls = mask_cls[keep]
    cur_mask_cls = cur_mask_cls[:, :-1]

    cur_prob_masks = cur_scores.view(-1, 1, 1) * cur_masks

    h, w = cur_masks.shape[-2:]
    panoptic_seg = torch.zeros((h, w), dtype=torch.int32, device=cur_masks.device)
    segments_info = []

    current_segment_id = 0

    if cur_masks.shape[0] == 0:
        # We didn't detect any mask :(
        return panoptic_seg, segments_info
    else:
        # take argmax
        cur_mask_ids = cur_prob_masks.argmax(0)
        stuff_memory_list = {}
        for k in range(cur_classes.shape[0]):
            pred_class = cur_classes[k].item()
            isthing = pred_class in metadata.thing_dataset_id_to_contiguous_id.values()
            mask_area = (cur_mask_ids == k).sum().item()
            original_area = (cur_masks[k] >= 0.5).sum().item()
            mask = (cur_mask_ids == k) & (cur_masks[k] >= 0.5)

            if mask_area > 0 and original_area > 0 and mask.sum().item() > 0:
                if mask_area / original_area < overlap_threshold:
                    continue

                # merge stuff regions
                if not isthing:
                    if int(pred_class) in stuff_memory_list.keys():
                        panoptic_seg[mask] = stuff_memory_list[int(pred_class)]
                        continue
                    else:
                        stuff_memory_list[int(pred_class)] = current_segment_id + 1

                current_segment_id += 1
                panoptic_seg[mask] = current_segment_id

                segments_info.append(
                    {
                        "id": current_segment_id,
                        "isthing": bool(isthing),
                        "category_id": int(pred_class),
                    }
                )

        return panoptic_seg, segments_info

def instance_inference(mask_cls, mask_pred):
    # mask_pred is already processed to have the same shape as original input
    image_size = mask_pred.shape[-2:]

    # [Q, K]
    scores = F.softmax(mask_cls, dim=-1)[:, :-1]
    labels = torch.arange(sem_seg_head.num_classes, device=device).unsqueeze(0).repeat(num_queries, 1).flatten(0, 1)
    # scores_per_image, topk_indices = scores.flatten(0, 1).topk(num_queries, sorted=False)
    scores_per_image, topk_indices = scores.flatten(0, 1).topk(test_topk_per_image, sorted=False)
    labels_per_image = labels[topk_indices]

    topk_indices = topk_indices // sem_seg_head.num_classes
    # mask_pred = mask_pred.unsqueeze(1).repeat(1, sem_seg_head.num_classes, 1).flatten(0, 1)
    mask_pred = mask_pred[topk_indices]

    # if this is panoptic segmentation, we only keep the "thing" classes
    if panoptic_on:
        keep = torch.zeros_like(scores_per_image).bool()
        for i, lab in enumerate(labels_per_image):
            keep[i] = lab in metadata.thing_dataset_id_to_contiguous_id.values()

        scores_per_image = scores_per_image[keep]
        labels_per_image = labels_per_image[keep]
        mask_pred = mask_pred[keep]

    result = Instances(image_size)
    # mask (before sigmoid)
    result.pred_masks = (mask_pred > 0).float()
    result.pred_boxes = Boxes(torch.zeros(mask_pred.size(0), 4))
    # Uncomment the following to get boxes from masks (this is slow)
    # result.pred_boxes = BitMasks(mask_pred > 0).get_bounding_boxes()

    # calculate average mask prob
    mask_scores_per_image = (mask_pred.sigmoid().flatten(1) * result.pred_masks.flatten(1)).sum(1) / (result.pred_masks.flatten(1).sum(1) + 1e-6)
    result.scores = scores_per_image * mask_scores_per_image
    result.pred_classes = labels_per_image
    return result

In [ ]:
processed_results = []
batched_inputs = dataset_dict["image"]

for mask_cls_result, mask_pred_result, input_per_image, image_size in zip(
    mask_cls_results, mask_pred_results, batched_inputs, images.image_sizes):
    height = 1024 # input_per_image.get("height", image_size[0])
    width = 1024 # input_per_image.get("width", image_size[1])
    processed_results.append({})

    if sem_seg_postprocess_before_inference:
        mask_pred_result = retry_if_cuda_oom(sem_seg_postprocess)(
            mask_pred_result, image_size, height, width
        )
        mask_cls_result = mask_cls_result.to(mask_pred_result)

    # semantic segmentation inference
    if semantic_on:
        r = retry_if_cuda_oom(semantic_inference)(mask_cls_result, mask_pred_result)
        if not sem_seg_postprocess_before_inference:
            r = retry_if_cuda_oom(sem_seg_postprocess)(r, image_size, height, width)
        processed_results[-1]["sem_seg"] = r

    # panoptic segmentation inference
    if panoptic_on:
        panoptic_r = retry_if_cuda_oom(panoptic_inference)(mask_cls_result, mask_pred_result)
        processed_results[-1]["panoptic_seg"] = panoptic_r

    # instance segmentation inference
    if instance_on:
        instance_r = retry_if_cuda_oom(instance_inference)(mask_cls_result, mask_pred_result)
        processed_results[-1]["instances"] = instance_r


In [ ]:
processed_results[0].keys()

dict_keys(['sem_seg', 'panoptic_seg', 'instances'])

In [ ]:
processed_results[0]['sem_seg'].shape

torch.Size([133, 1024, 1024])